In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

sns.set_style("darkgrid")
from tqdm import tqdm_notebook as tqdm

In [5]:
import sys

sys.path.insert(0, "..")
from fair_metrics.group_fairness import disparity_ratio, attributable_disparity


In [3]:
data = pd.read_csv("../row_data/UndergraduateAdmissionsData.csv")

/tmp/ipykernel_44208/942094448.py:1: DtypeWarning: Columns (6,12) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("../row_data/UndergraduateAdmissionsData.csv")


In [6]:
print("Disparity ratio", disparity_ratio(data, "accept", "gender", "M", "F"))
print(
    "Attributable disparity", attributable_disparity(data, "accept", "gender", "M", "F")
)

Disparity ratio 1.0318813523568249
Attributable disparity 0.01947447096701138


In [11]:
print(
    "Disparity ratio",
    disparity_ratio(data[data["accept"] == 1.0], "enroll", "gender", "M", "F"),
)
print(
    "Attributable disparity",
    attributable_disparity(data[data["accept"] == 1.0], "enroll", "gender", "M", "F"),
)

Disparity ratio 0.9082287848284822
Attributable disparity -0.04015640612927368


In [41]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 856914 entries, 0 to 856913
Data columns (total 15 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ACTCOMP        484947 non-null  float64
 1   SATCOMP        593083 non-null  float64
 2   HS_GPA         554682 non-null  float64
 3   gender         699261 non-null  object 
 4   ethnicity      803670 non-null  object 
 5   accepted       835324 non-null  object 
 6   enrolled       779185 non-null  object 
 7   year_incoming  842098 non-null  float64
 8   inst           856914 non-null  object 
 9   residency      856914 non-null  int64  
 10  SATV           390614 non-null  float64
 11  SATM           390638 non-null  float64
 12  athlete        241221 non-null  object 
 13  accept         835324 non-null  float64
 14  enroll         779185 non-null  float64
dtypes: float64(8), int64(1), object(6)
memory usage: 98.1+ MB


In [42]:
# Preprocess ethnicity
print("NAN for ethnicity", data[data["ethnicity"] == "U"].shape[0])
print("Unstated for ethnicity", data[pd.isna(data["ethnicity"])].shape[0])
# We can change Nan ethnicity to Unstated
data.loc[pd.isna(data["ethnicity"]), "ethnicity"] = "U"

NAN for ethnicity 26855
Unstated for ethnicity 53244


In [43]:
data["ethnicity"].value_counts()

ethnicity
W    504777
H    101000
U     80099
B     73621
A     71514
O      8887
I      8623
N      8393
Name: count, dtype: int64

In [44]:
## Preprocess athlete

print(data["athlete"].value_counts())
# Assume that 0.0, 0, N, ., No are NOT athlete values, encode them with 0.0
# Assume Y, 1, Yes, ATHLSCH, Athlete, t, 1.0, A are athelete values, encode them with 1.0
# Assume Nan are unstated, encode them with 0.5
yes_values = ["Y", "Yes", "1", "1.0", "ATHLSCH", "Athlete", "T", "A"]
no_values = ["N", "No", "0", "0.0", "."]
data["athlete"] = data["athlete"].replace(yes_values, 1.0).replace(no_values, 0.0)
data.loc[pd.isna(data["athlete"]), "athlete"] = 0.5

athlete
0.0        94371
0          58701
N          47758
.          32599
No          5640
Y            608
1            542
Yes          325
ATHLSCH      209
Athlete      192
T            185
1.0           69
A             22
Name: count, dtype: int64


In [45]:
data["athlete"].value_counts()

athlete
0.5    615693
0.0    239069
1.0      2152
Name: count, dtype: int64

In [46]:
# Preprocess Exam scores
# Values from 1 to 100, we assume that zeroes are same as missed values
print(data["ACTCOMP"].unique())
print("NAN for ACTCOMP", data[pd.isna(data["ACTCOMP"])].shape[0])
print("0 for ACTCOMP", data[data["ACTCOMP"] == 0].shape[0])
data.loc[pd.isna(data["ACTCOMP"]), "ACTCOMP"] = 0.0
print(data["ACTCOMP"].value_counts())

[ 16.    nan  18.   13.   20.   17.   28.   14.   21.   11.   12.   19.
  25.   22.   15.   26.   24.   23.   30.   29.   27.   31.   34.    8.
  32.   33.    9.   10.   35.    4.    5.   36.   18.5   0.    3.    1.
   7.   99.   38.   48.   40.   78.   44.   42.   92.   66.   50.   72.
  54.   46.   52.  100.   56.   76.   63.   58.   57.   69.   81.   62.
  88.   51.   60.   75.   64.   45.  104.   90.   47.    6.    2. ]
NAN for ACTCOMP 371967
0 for ACTCOMP 55138
ACTCOMP
0.0      427105
24.0      36380
25.0      35508
23.0      35423
22.0      33342
          ...  
92.0          1
18.5          1
100.0         1
99.0          1
2.0           1
Name: count, Length: 70, dtype: int64


In [47]:
data["SATCOMP"].value_counts()

SATCOMP
0.0       69487
1120.0     9534
1110.0     9504
1140.0     9450
1210.0     9384
          ...  
62.0          1
1066.0        1
56.0          1
46.0          1
1098.0        1
Name: count, Length: 271, dtype: int64

In [10]:
print(data["SATCOMP"].unique())
# Assume that zeroes are same as missed values
print("NAN for SATCOMP", data[pd.isna(data["SATCOMP"])].shape[0])
print("0 for SATCOMP", data[data["SATCOMP"] == 0].shape[0])
data.loc[pd.isna(data["SATCOMP"]), "SATCOMP"] = 0.0
print(data["SATCOMP"].value_counts())

[ 830.   nan  960. 1020. 1160. 1050.  700. 1350. 1100.  980.  890.  750.
  910.  770.  870. 1040.  880.  680.  800.  840.  580.  690. 1060.  820.
  950.  900.  710. 1230. 1030.  990.  920. 1000. 1130.  730.  810. 1220.
 1180.  940.  620. 1120. 1270.  970. 1200. 1010.  930. 1070.  590. 1240.
  790.  660.  720.  760. 1140.  670. 1210. 1190. 1170. 1150.  570.  780.
 1090. 1310.  860.  850. 1250. 1430. 1110.  640. 1260.  600.  740. 1290.
 1460.  650.  550. 1480. 1390. 1080. 1420. 1320.  630. 1380. 1340. 1500.
  530. 1280. 1218. 1330. 1360. 1490.  610. 1410. 1580.  470.  520.  440.
 1470.  560.  540.  410. 1300.  460.  450. 1370. 1062. 1560. 1510. 1400.
  400.  510.  500. 1550. 1450. 1440.  480. 1520.  490.  430. 1530. 1540.
 1042. 1590. 1600. 1066.  420. 1570. 1117.  781.  935. 1028.  768.  973.
  999. 1383.    0. 1017. 1237.  958. 1137. 1158.  991. 1052.  544. 1041.
  685. 1116.  126.  114.  112.  121.  127.  124.  134.  123.   95.  135.
  113.  125.   94.  129.  105.  108.  116.  107.  1

In [48]:
print(data["SATV"].unique())
# Assume that zeroes are same as missed values
print("NAN for SATV", data[pd.isna(data["SATV"])].shape[0])
print("0 for SATV", data[data["SATV"] == 0].shape[0])
data.loc[pd.isna(data["SATV"]), "SATV"] = 0.0
print(data["SATV"].value_counts())

[  nan  420.  600.  510.  450.  480.  430.  640.  400.  350.  380.  610.
  460.  590.  410.  440.  530.  490.  330.  570.  550.  540.  560.  500.
  470.  310.  660.  340.  670.  520.  580.  370.  630.  360.  390.  280.
  620.  770.  210.  690.  320.  270.  300.  200.  290.  260.  250.  700.
  710.  650.  790.  760.  730.  680.  750.  740.  800.  780.  240.  720.
  230.  220.  582.  489.  611.  519.    0.  397.  628.  498.  491. 1200.
 1000. 1100. 1110. 1530.  820.  880. 1120.  840. 1480. 1040. 1140.  920.
  940. 1260.   66.   54.   55.   56.   59.   60.   58.   75.   62.   64.
   50.   67.   53.   61.   49.   51.   48.   52.   63.   45.   72.   33.
   47.   44.   57.   76.   65.   68.   74.   46.   41.   43.   80.   42.
   69.   70.   39.   40.   71.   38.   36.   79.   73.   32.   30.   77.
   35.   78.   37.   34.   25.   24.   28.   22.   31.   26.   20.   23.
   21.   27.   29.  419.  436.  531.  571.]
NAN for SATV 466300
0 for SATV 42174
SATV
0.0       508474
530.0      13231
520.

In [12]:
print(data["SATM"].unique())
# Assume that zeroes are same as missed values
print("NAN for SATM", data[pd.isna(data["SATM"])].shape[0])
print("0 for SATM", data[data["SATM"] == 0].shape[0])
data.loc[pd.isna(data["SATM"]), "SATM"] = 0.0
print(data["SATM"].value_counts())

[  nan  420.  610.  600.  540.  570.  480.  470.  390.  380.  620.  490.
  560.  510.  440.  370.  590.  550.  460.  350.  400.  410.  520.  580.
  500.  450.  430.  340.  330.  670.  530.  680.  360.  300.  320.  290.
  740.  690.  260.  230.  650.  240.  630.  200.  270.  250.  640.  700.
  310.  280.  660.  710.  210.  760.  770.  750.  800.  730.  720.  780.
  790.  220.  577.  506.  361.  455.  518.  547.  368.  533.    0.  609.
  548.  820. 1180.  920. 1290. 1590.  960.  940. 1360. 1060. 1300. 1120.
 1380.  900. 1040. 1240.  860.   60.   57.   65.   68.   66.   59.   45.
   64.   54.   56.   63.   49.   61.   76.   62.   67.   50.   53.   51.
   44.   48.   69.   42.   55.   74.   58.   80.   70.   47.   38.   71.
   78.   41.   72.   52.   77.   46.   79.   34.   35.   73.   43.   28.
   75.   40.   39.   37.   33.   36.   29.   32.   31.   27.   26.   20.
   30.   22.   24.  565.  579.  569.  509.]
NAN for SATM 466276
0 for SATM 42174
SATM
0.0       508450
540.0      12217
570.

In [13]:
print(data["HS_GPA"].unique())
# Assume that zeroes are same as missed values
print("NAN for HS_GPA", data[pd.isna(data["HS_GPA"])].shape[0])
print("0 for HS_GPA", data[data["HS_GPA"] == 0].shape[0])
data.loc[pd.isna(data["HS_GPA"]), "HS_GPA"] = 0.0
print(data["HS_GPA"].value_counts())

[2.38    nan 2.87  ... 1.849 1.111 1.974]
NAN for HS_GPA 302232
0 for HS_GPA 10957
HS_GPA
0.000     313189
4.000      18013
3.000       8525
3.500       6585
3.300       3705
           ...  
95.460         1
12.090         1
11.360         1
12.290         1
1.974          1
Name: count, Length: 8416, dtype: int64


In [14]:
# Remove data without information about exams
data[
    (data["SATCOMP"] == 0)
    & (data["ACTCOMP"] == 0)
    & (data["SATM"] == 0)
    & (data["SATV"] == 0)
    & (data["HS_GPA"] == 0)
]

data = data[
    (data["SATCOMP"] != 0)
    | (data["ACTCOMP"] != 0)
    | (data["SATM"] != 0)
    | (data["SATV"] != 0)
    | (data["HS_GPA"] != 0)
]

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 823527 entries, 0 to 856913
Data columns (total 15 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ACTCOMP        823527 non-null  float64
 1   SATCOMP        823527 non-null  float64
 2   HS_GPA         823527 non-null  float64
 3   gender         675952 non-null  object 
 4   ethnicity      823527 non-null  object 
 5   accepted       802366 non-null  object 
 6   enrolled       746996 non-null  object 
 7   year_incoming  808879 non-null  float64
 8   inst           823527 non-null  object 
 9   residency      823527 non-null  int64  
 10  SATV           823527 non-null  float64
 11  SATM           823527 non-null  float64
 12  athlete        823527 non-null  float64
 13  accept         802366 non-null  float64
 14  enroll         746996 non-null  float64
dtypes: float64(9), int64(1), object(5)
memory usage: 100.5+ MB


In [16]:
print(data["gender"].value_counts())
print("Nan for Gender", data[pd.isna(data["gender"])].shape[0])
## Let's rows with missing gender
data = data[pd.notna(data["gender"])]
print("Nan for Gender", data[pd.isna(data["gender"])].shape[0])

gender
F    375976
M    299976
Name: count, dtype: int64
Nan for Gender 147575
Nan for Gender 0


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 675952 entries, 0 to 856913
Data columns (total 15 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ACTCOMP        675952 non-null  float64
 1   SATCOMP        675952 non-null  float64
 2   HS_GPA         675952 non-null  float64
 3   gender         675952 non-null  object 
 4   ethnicity      675952 non-null  object 
 5   accepted       654791 non-null  object 
 6   enrolled       599422 non-null  object 
 7   year_incoming  672050 non-null  float64
 8   inst           675952 non-null  object 
 9   residency      675952 non-null  int64  
 10  SATV           675952 non-null  float64
 11  SATM           675952 non-null  float64
 12  athlete        675952 non-null  float64
 13  accept         654791 non-null  float64
 14  enroll         599422 non-null  float64
dtypes: float64(9), int64(1), object(5)
memory usage: 82.5+ MB


In [18]:
## Check for enroll/accept feature
data[(data["enrolled"] == "N") & (data["enroll"] != 0.0)]

,ACTCOMP,SATCOMP,HS_GPA,gender,ethnicity,accepted,enrolled,year_incoming,inst,residency,SATV,SATM,athlete,accept,enroll


In [19]:
data[(data["enrolled"] != "N") & (data["enroll"] == 0.0)]

,ACTCOMP,SATCOMP,HS_GPA,gender,ethnicity,accepted,enrolled,year_incoming,inst,residency,SATV,SATM,athlete,accept,enroll


In [21]:
data[(data["accepted"] != "N") & (data["accept"] == 0.0)]

,ACTCOMP,SATCOMP,HS_GPA,gender,ethnicity,accepted,enrolled,year_incoming,inst,residency,SATV,SATM,athlete,accept,enroll


In [20]:
data[(data["accepted"] == "N") & (data["accept"] != 0.0)]

,ACTCOMP,SATCOMP,HS_GPA,gender,ethnicity,accepted,enrolled,year_incoming,inst,residency,SATV,SATM,athlete,accept,enroll


In [22]:
data = data.drop(columns=["accepted", "enrolled"])

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 675952 entries, 0 to 856913
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ACTCOMP        675952 non-null  float64
 1   SATCOMP        675952 non-null  float64
 2   HS_GPA         675952 non-null  float64
 3   gender         675952 non-null  object 
 4   ethnicity      675952 non-null  object 
 5   year_incoming  672050 non-null  float64
 6   inst           675952 non-null  object 
 7   residency      675952 non-null  int64  
 8   SATV           675952 non-null  float64
 9   SATM           675952 non-null  float64
 10  athlete        675952 non-null  float64
 11  accept         654791 non-null  float64
 12  enroll         599422 non-null  float64
dtypes: float64(9), int64(1), object(3)
memory usage: 72.2+ MB


In [24]:
data[
    ["accept", "enroll"]
].value_counts()  ## we assume, that accept can't be 0, if enroll 1

accept  enroll
1.0     0.0       225773
0.0     0.0       213725
1.0     1.0       159612
0.0     1.0          312
Name: count, dtype: int64

In [25]:
data = data[~((data["accept"] == 0.0) & (data["enroll"] == 1.0))]

In [27]:
data["year_incoming"].value_counts()

year_incoming
2006.0    306914
2005.0    184142
2007.0    149328
2004.0     17834
2008.0     13520
Name: count, dtype: int64

In [28]:
data.loc[pd.isna(data["year_incoming"]), "year_incoming"] = data[
    "year_incoming"
].mode()[0]

In [29]:
# Remove rows with Nan in accept
data = data[pd.notna(data["accept"])]

In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 654479 entries, 0 to 856913
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ACTCOMP        654479 non-null  float64
 1   SATCOMP        654479 non-null  float64
 2   HS_GPA         654479 non-null  float64
 3   gender         654479 non-null  object 
 4   ethnicity      654479 non-null  object 
 5   year_incoming  654479 non-null  float64
 6   inst           654479 non-null  object 
 7   residency      654479 non-null  int64  
 8   SATV           654479 non-null  float64
 9   SATM           654479 non-null  float64
 10  athlete        654479 non-null  float64
 11  accept         654479 non-null  float64
dtypes: float64(8), int64(1), object(3)
memory usage: 64.9+ MB


In [31]:
data.to_csv("../processed_data/UndergraduateAdmissionsData.csv", index=False)

### Fairness measure

In [36]:
# Check disparity based on gender
print("Disparity ratio", disparity_ratio(data, "accept", "gender", "M", "F"))
print(
    "Attributable disparity", attributable_disparity(data, "accept", "gender", "M", "F")
)

Disparity ratio 1.0302868467164048
Attributable disparity 0.018967166382021006
